# 神经网络

## 1、前言
前面两节讲了最基本的机器学习算法，线性回归和 logistic 回归，这一节将介绍传统机器学习里面最后一个算法 —— 神经网络，这也是深度学习的基石。所谓的深度学习，也可以理解为很深层的神经网络。

## 2、Neural Network
其实简单的神经网络说起来很简单，先放图为敬：
![](img/pt_3_1.jpg)

通过图片就能很简单的看出来，其实每一层网络所做的就是 y = W x X + b，只不过 W 的维数由 X 和输出维数决定，比如 X 是 10 维向量，想要输出的维数，也就是中间层的神经元的个数为 20，那么 W 的维数就是 20x10， b 的维数就是 20x1，这样输出的 y 的维数就是 20 。

中间层的维数可以自己设计，而最后一层输出的维数就是你的分类数目，比如我们等会要做的 MNIST 数据集是 10 个数字的分类，那么最后输出层的神经元就是 10 。

## 3、构建模型

有了前两节的经验，这一节的代码就很简单了，数据的导入和之前一样

### 3.1、导入包和库

In [1]:
# 引入相应的第三方库和包
import torch
from torch import nn, optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

### 3.2、设置超参数

In [6]:
# 设置超参数
batch_size = 32
learning_rate = 1e-2
num_epoches = 10

### 3.3、MNIST 数据集

In [10]:
# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# 构建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### 3.4、定义模型

In [11]:
# 定义简单的前馈神经网络
class Neuralnetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Neuralnetwork, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x


model = Neuralnetwork(28 * 28, 300, 100, 10)
if torch.cuda.is_available():
    model = model.cuda()

上面定义了三层神经网络，输入是 28x28 ，因为图片大小是 28x28，中间两个隐藏层大小分别是 300 和 100 ，最后是个 10 分类问题，所以输出层为 10 。

训练过程与之前完全一样，我就不再重复了，可以直接去 github 参看完整的代码。

这是迭代 10 次之后的输出结果，可以和上一节的 logistic 回归比较一下。

### 3.5、设置 loss 和优化方法

In [12]:
# 设置损失函数和优化方法
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

### 3.6、训练模型
接下来我们开始训练模型

In [13]:
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)
        if torch.cuda.is_available():
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(epoch + 1, num_epoches, running_loss / (batch_size * i), running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(train_dataset))))
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if torch.cuda.is_available():
            img = Variable(img, volatile=True).cuda()
            label = Variable(label, volatile=True).cuda()
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(test_dataset)), eval_acc / (len(test_dataset))))
    print()

epoch 1
**********


D:\anacondapy3.6\lib\site-packages\ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
D:\anacondapy3.6\lib\site-packages\ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1/10] Loss: 1.965253, Acc: 0.000000
[1/10] Loss: 1.515437, Acc: 0.000000
[1/10] Loss: 1.235038, Acc: 0.000000
[1/10] Loss: 1.058675, Acc: 0.000000
[1/10] Loss: 0.937107, Acc: 0.000000
[1/10] Loss: 0.852510, Acc: 0.000000
Finish 1 epoch, Loss: 0.835322, Acc: 0.000000


D:\anacondapy3.6\lib\site-packages\ipykernel_launcher.py:40: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
D:\anacondapy3.6\lib\site-packages\ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
D:\anacondapy3.6\lib\site-packages\ipykernel_launcher.py:44: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
D:\anacondapy3.6\lib\site-packages\ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Test Loss: 0.383529, Acc: 0.000000

epoch 2
**********
[2/10] Loss: 0.392071, Acc: 0.000000
[2/10] Loss: 0.376297, Acc: 0.000000
[2/10] Loss: 0.377588, Acc: 0.000000
[2/10] Loss: 0.373689, Acc: 0.000000
[2/10] Loss: 0.369380, Acc: 0.000000
[2/10] Loss: 0.365048, Acc: 0.000000
Finish 2 epoch, Loss: 0.362707, Acc: 0.000000
Test Loss: 0.321266, Acc: 0.000000

epoch 3
**********
[3/10] Loss: 0.335998, Acc: 0.000000
[3/10] Loss: 0.329198, Acc: 0.000000
[3/10] Loss: 0.330766, Acc: 0.000000
[3/10] Loss: 0.328959, Acc: 0.000000
[3/10] Loss: 0.324366, Acc: 0.000000
[3/10] Loss: 0.324112, Acc: 0.000000
Finish 3 epoch, Loss: 0.323317, Acc: 0.000000
Test Loss: 0.297635, Acc: 0.000000

epoch 4
**********
[4/10] Loss: 0.306849, Acc: 0.000000
[4/10] Loss: 0.309829, Acc: 0.000000
[4/10] Loss: 0.310651, Acc: 0.000000
[4/10] Loss: 0.308911, Acc: 0.000000
[4/10] Loss: 0.308998, Acc: 0.000000
[4/10] Loss: 0.306553, Acc: 0.000000
Finish 4 epoch, Loss: 0.305961, Acc: 0.000000
Test Loss: 0.287965, Acc: 0.000

## 4、小结
我们这里因为时间有限，就列出来了 10 次迭代的结果。但是从中也可以发现，准确率大大提高了，其实 logistic 回归可以看成简单的一层网络，从这里我们就可以看出来为什么多层网络比单层网络的效果要好，这也是为什么深度学习要叫深度的原因了。

下一节，我们介绍卷积神经网络 —— CNN，这个模型也是计算机视觉领域的王牌模型。